<a href="https://colab.research.google.com/github/thefr33radical/projects/blob/master/others/demyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import io
import pandas as pd
print("load sample trip data in time_stamps")
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['input_file.csv']))

load sample trip data in time_stamps


Saving input_file.csv to input_file.csv


In [ ]:
!pip install demyst-analytics

# PART I - DATA CLEANING & VALIDATION

In [38]:
from demyst.analytics import Analytics
analytics = Analytics(key='39efd730ccd6d4fa9c2a69b3f82086d6')
import re

class ETL(object):
    def __init__(self):
        pass

    def DataWrangle(self,df):
        
        print(len(df))
        df_cleaned=df.copy(deep=True)
       
        # Fill/handle missing values - NA

        # Remove Duplicate Rows
        df_cleaned.drop_duplicates(inplace=True,keep='first')
        print("Duplicate Records : ",len(df)-len(df_cleaned))

        df_cleaned["street"]=df_cleaned["street"].astype(str)
        #df_cleaned["street"]= df_cleaned["street"].map(lambda x: re.sub(r'\W+', '', x))

        # Remove invalid charecters and columns for state
        df_cleaned["state"] = df_cleaned["state"].astype(str)
        df_cleaned["state"]=df_cleaned["state"].apply(lambda x: str.upper(x))
        df_cleaned["state"]= df_cleaned["state"].map(lambda x: re.sub(r'\W+', '', x))
        df_cleaned["state"]= df_cleaned["state"].replace('NAN',None)
        
        # Remove invalid charecters and columns for city
        df_cleaned["city"] = df_cleaned["city"].astype(str)
        df_cleaned["city"]=df_cleaned["city"].apply(lambda x: str.lower(x))
        df_cleaned["city"]= df_cleaned["city"].map(lambda x: re.sub(r'\W+', '', x))

        # Remove invalid charecters and columns for post_code
        df_cleaned["postcode"]=df_cleaned["postcode"].astype(str)
        df_cleaned["postcode"]=df_cleaned["postcode"].replace("nan",None)
        df_cleaned["post_code"]=df_cleaned["postcode"]
        df_cleaned.drop(columns=["safety_flag","postcode"],inplace=True)
         
        return df_cleaned
      

    def Enrich(self,path):
        analytics.validate(input_dataframe)
if __name__=="__main__":
    obj=ETL()
    df_cleaned=obj.DataWrangle(df)
 

11229
Duplicate Records :  20


In [5]:
analytics.validate(df_cleaned)

Column,Status,Description
street,All Valid,All values in this column are good to go.
state,All Valid,All values in this column are good to go.
city,All Valid,All values in this column are good to go.
post_code,All Valid,All values in this column are good to go.


# PART II - DATA ENRICHMENT

In [35]:
analytics.search(df_cleaned,strict=True)

,city,state,street,post_code
Option 1,☒,☒,☒,☒


In [11]:
# Enrich data with pitneybowes_geocode_us_address
res=analytics.enrich_and_download(["pitneybowes_geocode_us_address"],df_cleaned,validate=False)
"""
pitneybowes_geocode_us_address
GeocodeUSAddress takes an address and returns latitude/longitude coordinates. 
GeocodeUSAddress also standardizes and validates addresses using data from the U.S. Postal Service. 
GeocodeUSAddress can also geocode intersections. Instead of entering a mailing address, you can enter and intersection such as "Pearl St. and 28th" and obtain the coordinates of the intersection.
"""

Verifying providers...
Starting enrichment...


This enrichment will use 112.09 credits of the 21581 credits your organization currently has.


Enrich Job ID: 27878


IntProgress(value=1, max=2)

HTML(value='Checking status...')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (30,31,68,71,117,118,119) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [61]:
# Cleaning columns for data Enrichment 1
l=list(res.columns)
l=list(map(lambda x :"p_"+x.split(".")[-1],l))
res.columns=l
print(res.columns)

Index(['p_city', 'p_post_code', 'p_state', 'p_street', 'p_row_id',
       'p_client_id', 'p_is_hit', 'p_additional_input_data',
       'p_address_line_1', 'p_address_line_2',
       ...
       'p_us_carrier_route_code', 'p_us_carrier_route_sort',
       'p_us_city_delivery', 'p_us_urban_name', 'p_usbc_check_digit',
       'p_usfips_county_number', 'p_usfips_state_code',
       'p_usfips_state_county_code', 'p_uslot_code', 'p_error'],
      dtype='object', length=122)


In [32]:
# Enrich data with hosted_l2_political
res2=analytics.enrich_and_download(["hosted_l2_political"],df_cleaned,validate=False)

Verifying providers...
Starting enrichment...


This enrichment will use 112.09 credits of the 21245 credits your organization currently has.


Enrich Job ID: 27887


IntProgress(value=1, max=2)

HTML(value='Checking status...')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [62]:
# Cleaning columns for data Enrichment 2
l2=list(res2.columns)
l2=list(map(lambda x :"l2_"+x.split(".")[-1],l2))
res2.columns=l2
print(res2.columns)

Index(['l2_city', 'l2_post_code', 'l2_state', 'l2_street', 'l2_row_id',
       'l2_client_id', 'l2_2001_state_house_district',
       'l2_2001_state_legislative_district', 'l2_2001_state_senate_district',
       'l2_2001_us_congressional_district',
       ...
       'l2_water_control_water_conservation',
       'l2_water_control_water_conservation_sub_district', 'l2_water_district',
       'l2_water_public_utility_district',
       'l2_water_public_utility_subdistrict', 'l2_water_replacement_district',
       'l2_water_replacement_sub_district', 'l2_water_sub_district',
       'l2_weed_district', 'l2_error'],
      dtype='object', length=724)


# PART III - SAVING FILES

In [37]:
#Download required files cleaned cv
df_cleaned.to_csv("data_cleaned.csv")
files.download("data_cleaned.csv")
#files.download("enrichment1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [84]:
#Download required files enriched 1 csv
import numpy as np
resx = res.replace([0,' ','NULL',""],np.nan)
resx = resx.loc[:, resx.isnull().mean() < .5]
print(len(resx.columns),len(res.columns))
resx.to_csv("data_enrich_pitnebowes.csv")
files.download("data_enrich_pitnebowes.csv")

76 122


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [85]:
#Download required files enriched 2
resx2 = res2.replace([0,' ','NULL',""],np.nan)
resx2 = resx2.loc[:, resx2.isnull().mean() < .5]
print(len(resx2.columns),len(res2.columns))
resx2.to_csv("data_enrich_l2_political.csv")
files.download("data_enrich_l2_political.csv")

256 724


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [86]:
# Save combined file as CSV
resx=resx.drop(columns=['p_city', 'p_post_code', 'p_state', 'p_street'],inplace=True)
res3=pd.concat([resx2,resx],axis=1)
res3.drop(columns=['l2_state'],inplace=True)
res3.to_csv("combined_enriched_data.csv")
files.download("combined_enriched_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [87]:
# Save Combined file as JSON
res3.to_json('combined_enriched_data.json', orient='records', lines=True)
files.download("combined_enriched_data.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [91]:
# Save combined file as parquet
res3["l2_post_code"]=res3["l2_post_code"].astype(str)
res3.to_parquet("combined_enriched_data.parquet",compression='gzip')
files.download("combined_enriched_data.parquet")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>